# Imports

In [15]:
from Transformation import Transformation
import numpy as np
from sklearn.neighbors import NearestNeighbors
import networkx as nx
import torch
from numpy import mean
from igraph import Graph as igraphGraph
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import scipy.spatial.distance
from scipy.spatial.distance import cdist
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon, LinearRing

# Input

In [16]:
transformation = Transformation()

user_number_triangles = 500   #à diminuer si le process est trop long
number_neigh_tri = 20

# Create objects
stl_file_path = "3d_models/stl/Handle.stl"
mesh_data = transformation.stl_to_mesh(stl_file_path)
graph = transformation.mesh_to_graph(mesh_data)

transformation.print_graph_properties(graph, display_graph=False, display_labels=False)

Number of nodes: 5999
Number of edges: 17991


In [17]:
if len(graph._node)<20:
    raise Exception("Input mesh does not have enough vertices. (More than 20 is needed)")

# Point Sampler

### DevConv

In [18]:
def relu(array):
    return np.maximum(array, 0)

def sigmoid(array):
    return 1 / (1 + np.exp(-array))

In [19]:
class DevConv():
    def __init__(self, graph, output_dimension):
        self.graph = graph
        self.list_node = list(graph._node)

        self.W_phi = np.random.random((output_dimension))      #change
        self.W_theta = np.array([0.1, 0.1, 0.1]).reshape(1, -1)  # change
    
    def forward(self, previous_inclusion_score, return_flatten=True):
        list_inc_score = np.zeros((len(self.list_node), len(self.W_phi)))                               #list of "output_dimension" for each "list_node" element
        for index_current_node, (current_node, dict_neigh) in enumerate(self.graph._adj.items()):       # for each node and its adjacency nodes
            neigh_nodes = np.array(list(dict_neigh.keys()))                                             # array of adjacency nodes
            diff = current_node - neigh_nodes                                                           # Compute the differences between current_node and all neighbor nodes   (x_i - x_j)
            neigh_distances = np.linalg.norm(self.W_theta * diff, axis=1)                               # Compute the norm for each vector difference  ||W_theta * (x_i - x_j)||
            list_inc_score[index_current_node] = self.W_phi * np.max(neigh_distances)                   # Add (W_phi * ||W_theta * (x_i - x_j)||) to the inclusion score list

        if len(previous_inclusion_score)==0:                            # return if no previous inclusion score
            if return_flatten:
                list_inc_score = list_inc_score.flatten()
            return list_inc_score
        
        if list_inc_score.shape[1]!=1:                                  # If inclusion score is not vector
            list_inc_score = np.mean(list_inc_score, axis=1)            # Mean the matrix for each node

        # array of array to array
        if len(list_inc_score.shape)==2:                 
            if list_inc_score.shape[1]==1:
                list_inc_score = list_inc_score.flatten()

        # Return the mean of previous and current inclusion score
        return np.mean(np.array([previous_inclusion_score, list_inc_score], dtype=np.float64), axis=0)
        

In [20]:
devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]))
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = relu(inclusion_score)
print(inclusion_score)

devconv = DevConv(graph, 1)
inclusion_score = devconv.forward(previous_inclusion_score=inclusion_score)
inclusion_score = sigmoid(inclusion_score)
print(inclusion_score)
print(inclusion_score.shape)

[1.16151182 1.1621876  1.16151182 ... 1.08132839 1.56963627 1.08132839]
[0.87188657 0.87239384 0.87188657 ... 0.81169704 1.17824439 0.81169703]
[0.65409237 0.65417623 0.65409237 ... 0.64407544 0.70285465 0.64407544]
(5999,)


### Multinomial Sampling

In [21]:
normalized_inclusion_score = inclusion_score / np.sum(inclusion_score)  # normalize for multinomial sampling
normalized_inclusion_score = np.round(normalized_inclusion_score, 8)    # round to remove float imprecision

number_throws = 500     # small:more randomness    |   big:less randomness
mult_sampling = np.random.multinomial(number_throws, normalized_inclusion_score)
print(mult_sampling)

[0 0 0 ... 0 0 0]


In [22]:
target_number_point = min(len(graph._node), user_number_triangles*3)   # number of points for the simplification

index_k_nodes = np.argpartition(mult_sampling, -target_number_point)[-target_number_point:]     # Take the k ("target_number_points") largest values given by the multinomial sampling 
list_k_nodes = np.array(graph)[index_k_nodes]                                # Take the selected nodes (list of list of 3)
list_k_nodes[:5]

array([[76.81192 ,  8.348472, 92.64003 ],
       [76.21072 ,  8.348472, 92.71918 ],
       [87.05576 ,  8.114116, 83.10944 ],
       [87.139175,  8.114116, 82.475876],
       [86.93931 ,  8.114116, 83.73778 ]], dtype=float32)

# KNN extended graph

In [23]:
"""
_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points

extended_graph = nx.Graph()
for index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node
    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates
    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)
        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples
        extended_graph.add_edge(*edge)                                      # Add the edge to the graph

transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)
"""

"\n_, indices = NearestNeighbors(n_neighbors=15).fit(list_k_nodes).kneighbors(list_k_nodes)        # KNN of the selected points\n\nextended_graph = nx.Graph()\nfor index_poly, poly in enumerate(indices):                                 # For the neighbors of the 'index_poly' node\n    current_node = tuple(list_k_nodes[poly[0]])                             # Tuple of the main node coordinates\n    for index_other_node in poly[1:]:                                       # For every neighbors of the main node (main node is the first index)\n        edge = current_node, tuple(list_k_nodes[index_other_node])          # Create an edge of two tuples\n        extended_graph.add_edge(*edge)                                      # Add the edge to the graph\n\ntransformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)\n"

In [24]:
def knn_and_extended_graph(list_k_nodes_i, k_neighbors=15):
    """
    Perform k-NN on the given points and construct an extended graph.

    Parameters:
    - list_k_nodes_i: array-like
        List of coordinates of the selected points.
    - k_neighbors: int, optional (default=15)
        Number of neighbors for k-NN.

    Returns:
    - extended_graph_i: networkx.Graph
        Extended graph.
    """

    # Perform k-NN on the selected points
    _, indices = NearestNeighbors(n_neighbors=k_neighbors).fit(list_k_nodes_i).kneighbors(list_k_nodes_i)

    # Construct the extended graph
    extended_graph_i = nx.Graph()
    for poly in indices:
        current_node = tuple(list_k_nodes_i[poly[0]])
        for index_other_node in poly[1:]:
            edge = current_node, tuple(list_k_nodes_i[index_other_node])
            extended_graph_i.add_edge(*edge)

    return extended_graph_i

extended_graph = knn_and_extended_graph(list_k_nodes)
transformation.print_graph_properties(graph=extended_graph, display_graph=False, display_labels=False)

Number of nodes: 1500
Number of edges: 11774


# Edge Predictor

In [25]:
devconv = DevConv(extended_graph, 64)
inclusion_score = devconv.forward(previous_inclusion_score=np.array([]), return_flatten=False)
inclusion_score.shape

(1500, 64)

In [26]:
"""
inclusion_score = [[f_1_1  , f_1_2  , ..., f_63_1  ],
                    ...,
                   [f_1_M-1, f_1_M-1, ..., f_63_M-1]]
M = number of points
64 = hidden dimensions
"""


f = np.mean(inclusion_score, axis=1)                            # Flatten the matrix of inclusion score 
wq = np.random.rand(64)
wk = np.random.rand(64)

In [27]:
wq_f = wq.reshape(-1, 1) * f            # Wq*f
wk_f = wk.reshape(-1, 1) * f            # Wq*f
S = np.exp(np.dot(wq_f.T, wk_f))        # e^((wq_f.T)*(wk_f))

nodes = list(extended_graph.nodes())                                                                                # list of nodes
node_indices = {node: index for index, node in enumerate(nodes)}                                                    # dict{node: index}
neighbors_indices = [[node_indices[neighbor] for neighbor in extended_graph.neighbors(node)] for node in nodes]     # List of list : [[neigh1 of node1, neigh2 of node1...], [neigh1 of node2, neigh2 of node2...]...]


for index_current_node, neighbors_index in enumerate(neighbors_indices):        # For each neighbors of the 'index_current_node' node
    sum_columns = np.sum(S[:, neighbors_index], axis=1)                         # Sum along the rows the values of the neighbors
    S[index_current_node] = S[index_current_node] / sum_columns                 # And divide the current node columns by the sume of the neighbors

### Sparse Attention

In [28]:
# S = S*np.random.choice([0, 1], size=S.shape)      # Add a random mask to emulate the 'sparse'

# Face Candidates

#### Inputs

In [29]:
adjacency = nx.adjacency_matrix(extended_graph)
# S = np.random.rand(target_number_point, target_number_point)
print(adjacency)
print(S)

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 7)	1
  (0, 8)	1
  (0, 9)	1
  (0, 10)	1
  (0, 11)	1
  (0, 12)	1
  (0, 13)	1
  (0, 14)	1
  (0, 15)	1
  (0, 49)	1
  (0, 1372)	1
  (0, 1497)	1
  (1, 0)	1
  (1, 2)	1
  (1, 3)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 7)	1
  :	:
  (1498, 540)	1
  (1498, 546)	1
  (1498, 1281)	1
  (1498, 1282)	1
  (1498, 1284)	1
  (1498, 1370)	1
  (1498, 1371)	1
  (1498, 1499)	1
  (1499, 518)	1
  (1499, 519)	1
  (1499, 520)	1
  (1499, 521)	1
  (1499, 522)	1
  (1499, 523)	1
  (1499, 526)	1
  (1499, 538)	1
  (1499, 539)	1
  (1499, 540)	1
  (1499, 546)	1
  (1499, 1281)	1
  (1499, 1282)	1
  (1499, 1284)	1
  (1499, 1370)	1
  (1499, 1371)	1
  (1499, 1498)	1
[[3.84619836e-02 4.26533209e-02 8.37280671e-02 ... 9.80071708e-03
  7.63827124e-02 7.64527468e-02]
 [3.47386086e+02 3.27804909e-02 8.39624572e-02 ... 6.55371941e-03
  7.92449033e-02 7.92938204e-02]
 [2.77798177e+00 9.59598728e-03 3.21716113e-02 ... 1.41762695e-04
  3.81989099e-02 3.81454147e-0

In [30]:
A_s = np.zeros((target_number_point,target_number_point))   # Init A_s to 0
A_s = np.matmul(np.matmul(S, adjacency.toarray()), S.T)     # A_s = S * A * S.T
A_s = A_s/A_s.max()                                         # Normalize

In [31]:
print(A_s)  # symmétrique
print(A_s.shape)

[[5.23164143e-18 2.22551583e-17 2.39208439e-18 ... 8.66889610e-16
  4.26924984e-17 4.23039718e-17]
 [2.22551583e-17 3.72018321e-17 7.50741269e-18 ... 2.80187707e-15
  6.11016890e-16 6.22283619e-16]
 [2.39208439e-18 7.50741269e-18 1.20375107e-18 ... 2.73713270e-16
  2.32371085e-17 2.31475914e-17]
 ...
 [8.66889610e-16 2.80187707e-15 2.73713270e-16 ... 1.94723187e-13
  2.81840729e-14 2.87518778e-14]
 [4.26924984e-17 6.11016890e-16 2.32371085e-17 ... 2.81840729e-14
  8.98165605e-16 9.00037507e-16]
 [4.23039718e-17 6.22283619e-16 2.31475914e-17 ... 2.87518778e-14
  9.00037507e-16 9.03685100e-16]]
(1500, 1500)


# Face Classifier

### TriConv

#### Inputs

In [32]:
igraph_g = igraphGraph(directed=extended_graph.is_directed()).from_networkx(extended_graph)
print(igraph_g.summary())

IGRAPH U--- 1500 11774 -- 
+ attr: _nx_name (v)


In [33]:
triangles_ids_igraph = np.array(igraph_g.cliques(min=3, max=3))
print(triangles_ids_igraph[:3])
print(len(triangles_ids_igraph))

[[ 909 1144 1145]
 [ 515  516  517]
 [ 302  332  334]]
33557


In [34]:
triangles = np.array(igraph_g.vs['_nx_name'])[triangles_ids_igraph]   #les triangles
# triangles = [list(map(tuple, row)) for row in triangles.tolist()]
print(triangles[:3])
print(len(triangles))

[[[76.01218   -4.4479885  3.1426833]
  [76.39445   -5.0095077  3.0894926]
  [76.312546  -5.0095077  2.9476252]]

 [[75.78252    5.0095077 82.55535  ]
  [75.85237    5.0095077 82.51254  ]
  [75.9199     5.0095077 82.46613  ]]

 [[77.49305    6.5098743  2.360639 ]
  [77.24371    6.0434117  2.9387176]
  [77.195564   6.0434117  2.822471 ]]]
33557


In [35]:
# Extract indices for each triangle
i, j, k = triangles_ids_igraph.T

# Extract probabilities using advanced indexing
A_s_ij = A_s[i, j]
A_s_ik = A_s[i, k]
A_s_jk = A_s[j, k]

# Calculate the barycenter probabilities
p_init = np.zeros(len(triangles))
p_init = (A_s_ij + A_s_ik + A_s_jk) / 3

print(p_init.shape)
print(p_init)

(33557,)
[1.16900765e-15 4.20226425e-16 4.53715978e-16 ... 5.58345252e-03
 6.18131988e-03 5.36221469e-03]


#### Calculate barycenter

In [36]:
barycenters = np.mean(triangles, axis=1)
print(barycenters.shape)
print(barycenters[:3])

(33557, 3)
[[76.23972   -4.822335   3.0599337]
 [75.85159    5.0095077 82.511345 ]
 [77.31077    6.198899   2.7072756]]


#### KNN Tri

In [37]:
_, indices_neigh_tri = NearestNeighbors(n_neighbors=number_neigh_tri).fit(barycenters).kneighbors(barycenters)          # Find k='number_neigh_tri' neighbors of each triangles based on theirs barycenters

Diff Barycenters

In [38]:
# Calculates the difference between each barycenters and their n='number_neigh_tri'-1 neighbors
barycenters_diff = np.subtract(barycenters[indices_neigh_tri[:, 0]][:, np.newaxis], barycenters[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire

print(barycenters_diff.shape)

(33557, 19, 3)


#### calculate e norm matrix

In [39]:
diff_vectors = list()

for triangle in triangles:
    e_ij = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[1]))    #Calculate the vectors for each edge of each triangles
    e_ik = np.linalg.norm(np.array(triangle[0]) - np.array(triangle[2]))
    e_jk = np.linalg.norm(np.array(triangle[1]) - np.array(triangle[2]))
    diff_vectors.append([e_ij, e_ik, e_jk])

diff_vectors = np.array(diff_vectors)
print(diff_vectors.shape)
print(diff_vectors[:2])

(33557, 3)
[[0.6813692  0.6660128  0.16381155]
 [0.08192835 0.16381034 0.0819381 ]]


#### Calculate r

In [40]:
max_diff_vectors = diff_vectors.max(axis=1)       # calculate t_n_max
min_diff_vectors = diff_vectors.min(axis=1)       # calculate t_n_min
max_diff_vectors.shape

(33557,)

In [41]:
max_diff_vectors_diff = np.subtract(max_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], max_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_max - t_m_max
print(max_diff_vectors_diff.shape)
min_diff_vectors_diff = np.subtract(min_diff_vectors[indices_neigh_tri[:, 0]][:, np.newaxis], min_diff_vectors[indices_neigh_tri[:, 1:]])   #Inverser la différence des barycentres si nécéssaire   # calculate t_n_min - t_m_min
print(min_diff_vectors_diff.shape)

(33557, 19)
(33557, 19)


In [42]:
r_matrix = np.zeros((len(triangles), number_neigh_tri-1, 5))

r_matrix[:, :, 0]   = min_diff_vectors_diff
r_matrix[:, :, 1]   = max_diff_vectors_diff
r_matrix[:, :, 2:5] = barycenters_diff

print(r_matrix.shape)  # nb_triangles, len(indices_neigh_tri), 5dim/triangles

# Here r_matrix[i,j] represent the relation between the triangles (n = i) and (m = indices_neigh_tri_array[i,j])

(33557, 19, 5)


#### Calculate f

In [43]:
# Multi Layer Perceptron (MLP) * 3 
f_final = p_init    # TODO


final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles
final_scores = final_scores.numpy()
print(final_scores.sum())
print(final_scores.shape)
print(final_scores)

0.9999999999998949
(33557,)
[2.97689133e-05 2.97689133e-05 2.97689133e-05 ... 2.99355915e-05
 2.99534943e-05 2.99289693e-05]


C:\Users\Arthur\AppData\Local\Temp\ipykernel_8064\474768578.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  final_scores = torch.nn.functional.softmax(torch.tensor(f_final))    #proba des triangles


# Simplified Mesh

In [44]:
selected_triangles_indexes = np.argpartition(final_scores, -int(target_number_point/3))[-int(target_number_point/3):] 
selected_triangles = np.array(triangles)[selected_triangles_indexes]
print(selected_triangles.shape) # number triangles, number points, number dimensions(x,y,z)
print(selected_triangles[0])

(500, 3, 3)
[[39.        -8.57404   88.01863  ]
 [39.        -4.871695  82.671524 ]
 [51.        -6.3966537 96.04341  ]]


In [45]:
simplified_final_graph = nx.Graph()
for index_poly, poly in enumerate(selected_triangles):
    for index_current_node in range(len(poly)):
        current_node = tuple(poly[index_current_node])
        for index_other_node in range(index_current_node+1, len(poly)):
            edge = current_node, tuple(poly[index_other_node])
            simplified_final_graph.add_edge(*edge)
            # if attribute do not exists
            if len(simplified_final_graph.nodes[current_node])==0:
                simplified_final_graph.nodes[current_node]['index_triangle'] = set()
            simplified_final_graph.nodes[current_node]['index_triangle'].add(index_poly)
            if len(simplified_final_graph.nodes[tuple(poly[index_other_node])])==0:
                simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'] = set()
            simplified_final_graph.nodes[tuple(poly[index_other_node])]['index_triangle'].add(index_poly)
            
transformation.print_graph_properties(graph=simplified_final_graph, display_graph=False, display_labels=False)

Number of nodes: 52
Number of edges: 238


In [46]:
simplified_final_mesh = transformation.graph_to_mesh(simplified_final_graph)

#Affichage
transformation.mesh_to_display_vtk(mesh_data)
transformation.mesh_to_display_vtk(simplified_final_mesh)

# Fonctions Loss

## Probabilistic Chamfer distance

Récupération des points de l'espace de départ P

In [101]:
def torch_d_P_Ps(p_y, x, y):
    """All Tensors in input"""
    # print(p_y.shape, x.shape, y.shape)

    expanded_x1 = x.unsqueeze(1)
    expanded_x2 = y.unsqueeze(0)
    distances = torch.norm(expanded_x1 - expanded_x2, dim=2)

    min_x = distances.min(dim=1).values
    min_y = distances.min(dim=0)

    first_term = torch.sum(torch.index_select(p_y, 0, min_y.indices) * min_y.values)
    second_term = torch.sum(min_x * p_y)

    return first_term + second_term


torch_d_P_Ps(torch.Tensor(normalized_inclusion_score), torch.Tensor(np.array(graph)), torch.Tensor(np.array(extended_graph)))

torch.Size([5999]) torch.Size([5999, 3]) torch.Size([1500, 3])


tensor(1.0998)

In [60]:
keys_input = np.array(graph)
print("Les points avant K-NN : ", keys_input)
print("dim points avant K-NN : ", len(keys_input))  #return 5999

Les points avant K-NN :  [[75.0252     -0.66553295  3.8       ]
 [75.01418    -0.49935842 16.7       ]
 [75.          0.          3.8       ]
 ...
 [51.         -8.57404    -6.9813724 ]
 [39.         -8.699333   -6.327254  ]
 [51.         -8.699333   -6.327254  ]]
dim points avant K-NN :  5999


Récupération des points de l'espace de départ Ps

In [61]:
keys_output = np.array(extended_graph)
print("sampled points : ", keys_output)
print("dim sample points : ", len(keys_output))  #return: 50

sampled points :  [[76.81192    8.348472  92.64003  ]
 [76.21072    8.348472  92.71918  ]
 [77.408165   8.348472  92.52952  ]
 ...
 [63.         7.755995  94.15747  ]
 [75.83298    3.8609922 81.519745 ]
 [75.66306    3.8609922 81.79702  ]]
dim sample points :  1500


Distance Matrix

In [62]:
def matrix_distance_euclidienne_barycentres(barycentre_b, barycentre_b_hat):
    """
    Calculate distances between 2 set of points

    Parameters:
    - b_hat: array-like, shape (M, 3)
        Barycenters after k-NN.
    - number_neigh: int, optional (default=3)
        Number of neighbors for k-NN.

    Returns:
    - distances_after: array-like, shape (M, M)
        Matrix of distances between barycenters after the k-NN process.
    """
    return cdist(barycentre_b, barycentre_b_hat, metric='euclidean').T

distances_matrix = matrix_distance_euclidienne_barycentres(keys_input, keys_output)
print("Squared Euclidean Distances Matrix Shape:", distances_matrix.shape)
print("Distances Matrix:\n", distances_matrix)




Squared Euclidean Distances Matrix Shape: (1500, 5999)
Distances Matrix:
 [[ 89.31402785  76.4748585   89.24982328 ... 104.29309865 107.30746574
  103.68898746]
 [ 89.38276019  76.5416944   89.3184354  ... 104.22164468 107.17024102
  103.61661725]
 [ 89.2180374   76.38148303  89.15393223 ... 104.3368579  107.4172991
  103.73369841]
 ...
 [ 91.54234038  78.81720054  91.48020538 ... 103.14909416 104.61337238
  102.52783857]
 [ 77.85564003  64.97139608  77.82004795 ...  92.75642591  96.08082623
   92.14951825]
 [ 78.13086093  65.24611568  78.09533945 ...  92.97580146  96.26956733
   92.36837165]]


In [ ]:
# Calculer le min des x de distances_matrix
def min_pour_x_list(distances_matrix, x_in):
    return np.min(distances_matrix[:, x_in])

# Calculer le min des y de distances_matrix
def min_pour_y_list(distances_matrix, y_out):
    return np.min(distances_matrix[y_out, :])


In [ ]:
def d_P_Ps(keys_x, keys_y, p_y):
    """
    Calculate the Probabilistic Chamfer distance between two sets of points.

    Parameters:
    - keys_x (list): The input vertex set, a list of tuples.
    - keys_y (list): The sampled points, a list of tuples.
    - p_y (list): Their respective probabilities, a weight associated with each point in keys_y.

    Returns:
        chamfer_distance (float): The Probabilistic Chamfer distance between the two sets of points.
    """
    min_for_x = [p_y[i] * min_pour_x_list(distances_matrix, i) for i in range(len(keys_x))]
    min_for_y = [p_y[j] * min_pour_y_list(distances_matrix, j) for j in range(len(keys_y))]
    
    sum_for_y = np.sum(min_for_y)
    sum_for_x = np.sum(min_for_x)

    chamfer_distance = sum_for_y + sum_for_x
    
    return chamfer_distance


In [ ]:
print("PROBA : ", normalized_inclusion_score)

PROBA :  [0.00019197 0.00019199 0.00019197 ... 0.00018968 0.00020335 0.00018968]


In [ ]:
prob_chamfer_dist = d_P_Ps(keys_input, keys_output, normalized_inclusion_score)
print("Chamfer Distance:", prob_chamfer_dist)

Chamfer Distance: 4.255793966009449


## Probabilistic Surfaces Distance

### d_f_S_Ss

In [ ]:
# Creation igraph
igraph_g_original = igraphGraph(directed=extended_graph.is_directed()).from_networkx(graph)
print(igraph_g_original.summary())

#Find triangles
triangles_ids_igraph_original = np.array(igraph_g_original.cliques(min=3, max=3))
triangles = np.array(igraph_g_original.vs['_nx_name'])[triangles_ids_igraph_original]
print("triangles shape : ", triangles.shape)
print(triangles[0])

# Calculate barycenters
b = np.mean(triangles, axis=1)
print("shape b : ", b.shape)
print("Barycentre b: ", b)

IGRAPH U--- 5999 17991 -- 
+ attr: _nx_name (v), index_triangle (v)
triangles shape :  (11994, 3, 3)
[[39.        8.793697 -4.666995]
 [51.        8.793697 -4.666995]
 [39.        8.793697 -5.333005]]
shape b :  (11994, 3)
Barycentre b:  [[43.          8.793697   -4.8889985 ]
 [55.          1.0515273   3.7313068 ]
 [55.         -8.515905   -2.804371  ]
 ...
 [75.05613    -0.94137865 21.        ]
 [75.01313    -0.3882971   8.099999  ]
 [75.03516    -0.72036856 25.300001  ]]


In [ ]:
# Get finals selected triangles & calculate barycenters
b_hat = np.mean(selected_triangles, axis=1)
print(b_hat.shape)
print("B_HAT : ", b_hat)

(500, 3)
B_HAT :  [[86.61672    2.0560768 25.300001 ]
 [81.23691    2.6044734 25.300001 ]
 [71.33375   -8.525946  88.80524  ]
 ...
 [47.        -4.232605  91.3241   ]
 [55.        -8.294961  87.62905  ]
 [63.        -7.632799  86.746086 ]]


In [ ]:
distances = matrix_distance_euclidienne_barycentres(b, b_hat)
print(distances.shape)
print("Distances euclidiennes entre les barycentres b et b_hat :\n", distances)


(500, 11994)
Distances euclidiennes entre les barycentres b et b_hat :
 [[2859.18967864 1465.83481259 1901.23966356 ...  161.12205128
   436.45836812  141.84131636]
 [2411.74332409 1155.99555691 1601.8939027  ...   69.2650992
   343.53214384   49.51629803]
 [9881.38103548 7596.09287683 8659.11168608 ... 4668.93192118
  6593.09452021 4107.54241565]
 ...
 [9442.64423827 7764.41889398 8942.51511236 ... 5743.45714458
  7725.76452932 5157.48698186]
 [8995.61176704 7126.18836066 8178.25269021 ... 4895.75396666
  6787.91079053 4343.69241805]
 [9066.81852243 7030.87111229 8084.06426836 ... 4512.67320588
  6382.00505878 3968.24804145]]


In [ ]:
minimums_b = np.min(distances, axis=1)
print(minimums_b.shape)
print(minimums_b[:5])

(500,)
[28.17643517 26.45724819  0.19258887 58.67391697 60.97788051]


In [ ]:
d_f_S_Ss = np.sum(selected_triangles_indexes * minimums_b)
print(d_f_S_Ss)

188058353.55402204


### d_r_S_Ss

In [ ]:
distances = distances
min_d = distances.min(axis=1)
min_d.shape

(500,)

In [ ]:
p_y = np.array(final_scores)[selected_triangles_indexes]
p_y.shape

(500,)

In [ ]:
first_term = p_y * min_d
first_term.shape

(500,)

In [ ]:
number_neigh_selected_barycenters = min(50, len(b))
_, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=number_neigh_selected_barycenters).fit(b).kneighbors(b)

In [ ]:
indexes_neigh_selected_barycenters

array([[    0,  3197,  3204, ...,  3209,    67,  2784],
       [    1,  3307,  2916, ...,  3252,    72,    71],
       [    2,  2978,  3427, ...,    81,  3300,  3297],
       ...,
       [11991, 11988, 11165, ..., 11201, 11355, 11365],
       [11992,  2563,  2966, ...,  6500,  6502,  6495],
       [11993,  2965,  2564, ...,  2576, 11359, 11195]], dtype=int64)

In [ ]:
xt = b[indexes_neigh_selected_barycenters[:, 1:]].reshape((len(b)*(number_neigh_selected_barycenters-1),3))
print(xt.shape)

y = b_hat
print(y.shape)

(587706, 3)
(500, 3)


In [ ]:
distances_barycenter_knn = cdist(xt, y, metric='sqeuclidean')
distances_barycenter_knn.shape

(587706, 500)

In [ ]:
pt = final_scores[indexes_neigh_selected_barycenters[:, 1:]].reshape((len(b)*(number_neigh_selected_barycenters-1)))
pt.shape

(587706,)

In [ ]:
ptk_norm = pt*distances_barycenter_knn.T
print(ptk_norm.shape)

sum_ptk_norm = np.sum(ptk_norm, axis=1)
print(sum_ptk_norm.shape)

second_term = (1 - p_y) * (1/(number_neigh_selected_barycenters-1)) * sum_ptk_norm
print(second_term.shape)

(500, 587706)
(500,)
(500,)


In [ ]:
print(first_term.shape, second_term.shape)

d_r_S_Ss = np.sum(first_term + second_term)
d_r_S_Ss

(500,) (500,)


847267.1001266912

## Triangle Collision Loss

In [ ]:
def compute_lc_le_lo(p_t, m_c_e_o, Fs):
    """
    Compute the collision loss term L_c.

    Parameters:
    - p_t: 1D numpy array containing the probabilities of each triangle (indices)
    - m_c_t: 2D numpy array containing the number of faces penetrated by each triangle
    - Fs: 3D numpy array representing the vertices of triangles

    Returns:
    - L_c: Collision loss term
    """
    assert len(p_t) == len(m_c_e_o), "Input arrays must have the same length"

    penalty_per_triangle = p_t * m_c_e_o

    # Sum the penalties for all selected triangles
    total_penalty = np.sum(penalty_per_triangle)

    # Compute the collision loss term L_c
    L_c_e_o = (1 / len(Fs)) * total_penalty

    return L_c_e_o

# Example usage:
# Replace the arrays below with your actual data
# p_t = selected_triangles_indexes
# m_c_t = numpy array containing the number of faces penetrated by each triangle
# Fs = 3D numpy array representing the vertices of triangles
p_t = selected_triangles_indexes
Fs = triangles  # Given data

In [ ]:
number_neigh_barycenters = min(50, len(b_hat))
_, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=number_neigh_barycenters).fit(b_hat).kneighbors(b_hat)

### Lc

In [ ]:
mc = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    polygons_others_tri = MultiPolygon([Polygon(others_triangle) for others_triangle in others_triangles]).buffer(0)    # buffer 0 to correct invalid polygons => take the exterior of the shape


    intersection = lines_current_triangle.intersection(polygons_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        mc[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        mc[index_neigh_barycenters[0]] += 1
L_c = compute_lc_le_lo(p_t, mc, Fs)
print("L_c : ", L_c)

L_c :  2816.994580623645


### Le

In [ ]:
me = np.zeros((500))

for index_neigh_barycenters in indexes_neigh_selected_barycenters:
    current_triangle = selected_triangles[index_neigh_barycenters[0]]
    others_triangles = selected_triangles[index_neigh_barycenters[1:]]

    lines_current_triangle = LinearRing(current_triangle)
    lines_others_tri = MultiLineString([LineString([other_tri[0], other_tri[1], other_tri[2], other_tri[0]]) for other_tri in others_triangles])

    intersection = lines_current_triangle.intersection(lines_others_tri)
    if intersection.is_empty:
        continue
    if intersection.geom_type == 'MultiLineString':
        me[index_neigh_barycenters[0]] = len(intersection.geoms)
    else:
        me[index_neigh_barycenters[0]] += 1
L_e = compute_lc_le_lo(p_t, me, Fs)
print("L_e : ", L_e)

L_e :  23092.200100050024


### Lo

In [ ]:
mo = np.zeros((500))
L_o = 0

## Final Loss

In [ ]:
lambda_c, lambda_e, lambda_o = 0.01, 0.01, 0.01


Loss_L = prob_chamfer_dist + d_f_S_Ss + d_r_S_Ss + (lambda_c * L_c) + (lambda_e * L_e) + (lambda_o * L_o)
print(Loss_L)

188905884.0018895


Lo - échantillonnage de 100 points à partir de chaque triangle

In [ ]:

def sample_points_from_triangle(t, num_points=100):
    v1, v2, v3 = t
    bary_coords = np.random.rand(num_points, 2)
    sqrt_bary_coords = np.sqrt(bary_coords[:, 0])

    u = sqrt_bary_coords
    v = bary_coords[:, 1]

    """
    La formule spécifique est dérivée de l'expression générale d'interpolation barycentrique 
    sommets A, B et C
    coord barycentriques: u et v
    coord cartésiennes: x,y,z 
    """
    x_coords = (1 - u - v) * v1[0] + u * v2[0] + v * v3[0]
    y_coords = (1 - u - v) * v1[1] + u * v2[1] + v * v3[1]
    z_coords = (1 - u - v) * v1[2] + u * v2[2] + v * v3[2]

    sampled_points = np.column_stack((x_coords, y_coords, z_coords))
    return sampled_points

points100 = sample_points_from_triangle(triangle, num_points=100)
points100
points100.shape

Les Aires

In [ ]:
number_neigh_selected_barycenters = min(50, len(b_hat))
def knnbar(nn):
  _, indexes_neigh_selected_barycenters = NearestNeighbors(n_neighbors=nn).fit(b_hat).kneighbors(b_hat)
  return _, indexes_neigh_selected_barycenters

In [ ]:
d, indices = knnbar(number_neigh_selected_barycenters)
indices

In [ ]:
from scipy.spatial import distance as scipy_distance

def calculate_triangle_area(triangle):
    # Fonction pour calculer l'aire d'un triangle en utilisant la formule de Héron
    side_lengths = [scipy_distance.euclidean(triangle[i], triangle[(i + 1) % 3]) for i in range(3)]
    s = sum(side_lengths) / 2
    return np.sqrt(s * np.prod([s - length for length in side_lengths]))

def penalize_overlapping_triangles(points_list, triangles, k=50):
    # Fonction pour pénaliser les triangles qui se chevauchent
    assignment_results = []
    overlapping_triangles = np.zeros(len(triangles), dtype=int)  # Déclaration en dehors de la boucle

    # ajustement de la valeur maximale de k en fonction du nombre de triangles
    k = min(k, len(triangles))

    # NearestNeighbors pour trouver les k triangles les plus proches pour chaque point
    knn_model = NearestNeighbors(n_neighbors=k).fit(np.vstack(triangles))
    d, indices = knn_model.kneighbors(np.array(points_list)[:, 0, :])

    for i, point_list in enumerate(points_list):
        point = point_list[0]

        # Verif que les indices sont valides
        closest_triangle_indices = indices[i, :k]
        closest_triangle_indices = closest_triangle_indices[closest_triangle_indices < len(triangles)]

        # Accéder aux triangles en utilisant les indices valides
        valid_triangle_indices = []
        for idx in closest_triangle_indices:
            modified_triangle = np.copy(triangles[idx])
            modified_triangle[0] = point

            area_original = calculate_triangle_area(triangles[idx])
            area_modified = calculate_triangle_area(modified_triangle)

            # Vérifier si la somme des aires est proche de l'aire du triangle
            if not np.isclose(area_original, area_modified, rtol=1e-5):
                assignment_results.append((point, triangles[idx]))
                valid_triangle_indices.append(idx)

        # MAJ du overlapping_triangles avec les indices valides
        overlapping_triangles[valid_triangle_indices] += 1

    penalties = overlapping_triangles
    total_penalty = np.sum(penalties)
    Lo = (1 / len(triangles)) * total_penalty

    return Lo


L_o = penalize_overlapping_triangles([points100], triangles, k=50)
print("Pénalité pour les triangles qui se chevauchent :", L_o)
